In [9]:
# Import findsparck e inicializa
import findspark
findspark.init()

In [18]:
import pyspark
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from pyspark import SparkConf, SparkContext

from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer, MinMaxScaler, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml.stat import Correlation
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

In [23]:
# Formatação das saídas, pois a saída do pyspark ocupa muito espaço

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 400)
from matplotlib.axes._axes import _log as matrplotlib_axes_logger
matrplotlib_axes_logger.setLevel('ERROR')

# Preparando o Ambiente Spark

In [27]:
# Definindo semente aleatória (seed) para reprodutibilidade do notebook
rnd_seed = 23
np.random.seed = rnd_seed
np.random.set_state = rnd_seed

In [28]:
# Criando o Sparnk Contex
sc = SparkContext(appName = 'Mini_Projeto3')

In [29]:
# Criando a sessão Spark
spark_session = SparkSession.Builder().getOrCreate()

# Quando não identificamos o cluster, automaticamente é logado no local

In [30]:
# Visualizando o objeto spark_session
spark_session

## Carregamento de Conjunto de Dados

In [35]:
# Carregando os dados a partir da sessão Spark
df_spark = spark_session.read.csv("C:/formacao_dataScience_DSA_DADOS/01_bigData_RealTime_Python_Spark/cap11_Apache_Spark_Processamento/dados/dataset.csv",
                                 header = 'true', inferSchema = 'true') #Detectar o padrão dos dados

In [36]:
# Tipo do objeto
type(df_spark)

pyspark.sql.dataframe.DataFrame

In [37]:
# Visualização dos dados
df_spark.show()

+----------+----+----+----+-----+------------+-----------------+--------------+
| Timestamp|Open|High| Low|Close|Volume_(BTC)|Volume_(Currency)|Weighted_Price|
+----------+----+----+----+-----+------------+-----------------+--------------+
|1325317920|4.39|4.39|4.39| 4.39|  0.45558087|     2.0000000193|          4.39|
|1325319300| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325319360| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325319420| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325319480| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325319540| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325319600| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325319660| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325345040| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325345100| NaN| NaN| NaN|  NaN|       

In [38]:
# Visualizar os metadados (Schema)
df_spark.printSchema()

root
 |-- Timestamp: integer (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume_(BTC): double (nullable = true)
 |-- Volume_(Currency): double (nullable = true)
 |-- Weighted_Price: double (nullable = true)



In [39]:
df_spark.count()

4856600